In [ ]:
from mtcnn.mtcnn import MTCNN
import cv2
import os
from google.colab.patches import cv2_imshow

detector = MTCNN()

# Create an 'images' folder if it doesn't exist
if not os.path.exists('images'):
    os.makedirs('images')

# Specify the folder path where videos are located
video_folder = './drive/MyDrive/video/'

# Get a list of all video files with various extensions in the folder
video_files = [f for f in os.listdir(video_folder) if f.lower().endswith(('.mp4', '.avi', '.mov'))]

# Loop through each video file
for video_file in video_files:
    video_path = os.path.join(video_folder, video_file)

    # Create a folder with the same name as the video (without extension) in 'images'
    video_name = os.path.splitext(video_file)[0]
    image_folder = os.path.join('images', video_name)
    os.makedirs(image_folder, exist_ok=True)

    # Open the video file for processing
    cap = cv2.VideoCapture(video_path)

    prev_num_faces = 0
    frame_number = 0

    # Motion detection variables
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fgbg = cv2.createBackgroundSubtractorMOG2()

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Apply motion detection
        fgmask = fgbg.apply(frame)
        motion_area = cv2.countNonZero(fgmask)

        faces = detector.detect_faces(frame)
        num_faces = len(faces)

        # If there's significant motion or a change in the number of faces, save the frame
        if motion_area > 1000 or num_faces != prev_num_faces:
            cv2.imwrite(os.path.join(image_folder, f'frame_{frame_number}.jpg'), frame)
            frame_number += 1

        for face in faces:
            x, y, w, h = face['box']
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        cv2_imshow(frame)

        prev_num_faces = num_faces

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


**Effecient code**

In [ ]:
from mtcnn.mtcnn import MTCNN
import cv2
import os
from google.colab.patches import cv2_imshow

detector = MTCNN()

# Create an 'images' folder if it doesn't exist
if not os.path.exists('images'):
    os.makedirs('images')

# Specify the folder path where videos are located
video_folder = './drive/MyDrive/video/'

# Get a list of all video files with various extensions in the folder
video_files = [f for f in os.listdir(video_folder) if f.lower().endswith(('.mp4', '.avi', '.mov'))]

# Loop through each video file
for video_file in video_files:
    video_path = os.path.join(video_folder, video_file)

    # Create a folder with the same name as the video (without extension) in 'images'
    video_name = os.path.splitext(video_file)[0]
    image_folder = os.path.join('images', video_name)
    os.makedirs(image_folder, exist_ok=True)

    # Open the video file for processing
    cap = cv2.VideoCapture(video_path)

    prev_num_faces = 0
    frame_number = 0
    frame_count = 0
    current_batch = []  # List to store frames in the current batch
    fgbg = cv2.createBackgroundSubtractorMOG2()

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Append the frame to the current batch
        current_batch.append(frame)
        frame_count += 1

        # Only process frames at a specific interval (e.g., every 5 frames)
        if frame_count % 5 != 0:
            continue

        # Apply motion detection
        fgmask = fgbg.apply(frame)
        motion_area = cv2.countNonZero(fgmask)

        faces = detector.detect_faces(frame)
        num_faces = len(faces)

        # If there's significant motion or a change in the number of faces, save the batch
        if motion_area > 1000 or num_faces != prev_num_faces:
            for batch_frame in current_batch:
                cv2.imwrite(os.path.join(image_folder, f'frame_{frame_number}.jpg'), batch_frame)
                frame_number += 1
            current_batch = []  # Reset the batch

        prev_num_faces = num_faces

        for face in faces:
            x, y, w, h = face['box']
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        cv2_imshow(frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
